In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [16]:
file = open('Requirements.txt', 'r')

lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines) 

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces 
data = data.split()
data = ' '.join(data)
data[:500]

'The Game must be developed using the Unity Game Engine,Game Engine,Ubiquitous,The game shall be developed using unity game engine,,ubiquitous,The game shall be developed using unity game engine,Game,Mobile app The Game must be programmed using the C language and its libraries compatible with the Unity Engine,Language,Ubiquitous,The game shall be written in C# and use Unity Engine-compatible libraries.,,ubiquitous,The game shall be written in C# and use Unity Engine-compatible libraries.,Game,Mob'

In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]


[1, 2, 9, 7, 19, 10, 1, 14, 2, 11, 2, 11, 4, 1, 2]

In [18]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

63


In [19]:
sequences = []

for i in range(10, len(sequence_data)):
    words = sequence_data[i-10:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]


The Length of sequences are:  313


array([[ 1,  2,  9,  7, 19, 10,  1, 14,  2, 11,  2],
       [ 2,  9,  7, 19, 10,  1, 14,  2, 11,  2, 11],
       [ 9,  7, 19, 10,  1, 14,  2, 11,  2, 11,  4],
       [ 7, 19, 10,  1, 14,  2, 11,  2, 11,  4,  1],
       [19, 10,  1, 14,  2, 11,  2, 11,  4,  1,  2],
       [10,  1, 14,  2, 11,  2, 11,  4,  1,  2,  5],
       [ 1, 14,  2, 11,  2, 11,  4,  1,  2,  5,  7],
       [14,  2, 11,  2, 11,  4,  1,  2,  5,  7, 19],
       [ 2, 11,  2, 11,  4,  1,  2,  5,  7, 19, 10],
       [11,  2, 11,  4,  1,  2,  5,  7, 19, 10, 14]])

In [20]:
X = []
y = []

for i in sequences:
    X.append(i[0:10])
    y.append(i[10])
    
X = np.array(X)
y = np.array(y)

In [21]:
print("Data: ", X[:10])
print("Response: ", y[:10])

Data:  [[ 1  2  9  7 19 10  1 14  2 11]
 [ 2  9  7 19 10  1 14  2 11  2]
 [ 9  7 19 10  1 14  2 11  2 11]
 [ 7 19 10  1 14  2 11  2 11  4]
 [19 10  1 14  2 11  2 11  4  1]
 [10  1 14  2 11  2 11  4  1  2]
 [ 1 14  2 11  2 11  4  1  2  5]
 [14  2 11  2 11  4  1  2  5  7]
 [ 2 11  2 11  4  1  2  5  7 19]
 [11  2 11  4  1  2  5  7 19 10]]
Response:  [ 2 11  4  1  2  5  7 19 10 14]


In [22]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
 

In [23]:
model = Sequential([
    Embedding(vocab_size, 10, input_length=10),
    LSTM(1000, return_sequences=True),
    LSTM(1000),
    Dense(1000, activation="relu"),
    Dense(vocab_size, activation="softmax")
])


In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 10, 10)            630       
                                                                 
 lstm_2 (LSTM)               (None, 10, 1000)          4044000   
                                                                 
 lstm_3 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense_2 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_3 (Dense)             (None, 63)                63063     
                                                                 
Total params: 13,112,693
Trainable params: 13,112,693
Non-trainable params: 0
_________________________________________________________________


In [25]:
# %pip install pydot

# # compile model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # plot model
# plot_model(model, to_file='./plot.png', show_layer_names=True)


In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=70, batch_size=64, callbacks=[checkpoint])

Epoch 1/70
5/5 [==============================] - ETA: 0s - loss: 4.0707
Epoch 1: loss improved from inf to 4.07068, saving model to next_words.h5
5/5 [==============================] - 8s 800ms/step - loss: 4.0707
Epoch 2/70
5/5 [==============================] - ETA: 0s - loss: 3.9195
Epoch 2: loss improved from 4.07068 to 3.91951, saving model to next_words.h5
5/5 [==============================] - 4s 804ms/step - loss: 3.9195
Epoch 3/70
5/5 [==============================] - ETA: 0s - loss: 3.9183
Epoch 3: loss improved from 3.91951 to 3.91832, saving model to next_words.h5
5/5 [==============================] - 4s 778ms/step - loss: 3.9183
Epoch 4/70
5/5 [==============================] - ETA: 0s - loss: 3.7879
Epoch 4: loss improved from 3.91832 to 3.78791, saving model to next_words.h5
5/5 [==============================] - 4s 781ms/step - loss: 3.7879
Epoch 5/70
5/5 [==============================] - ETA: 0s - loss: 3.7592
Epoch 5: loss improved from 3.78791 to 3.75921, saving 

In [27]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [28]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
       

['made', 'in', '']
1/1 [==============================] - 1s 947ms/step
to
['game', '']
1/1 [==============================] - 0s 31ms/step
to
['made', '']
Error occurred:  in user code:

    File "c:\Users\WB\anaconda3\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\WB\anaconda3\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\WB\anaconda3\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\WB\anaconda3\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "c:\Users\WB\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\WB\